# **Exchange in 2 dots**

The goal here is to reproduce the results from Matlab code.

# Loading modules and defining main variables

In [ ]:
import os, sys
sys.path.append(os.path.dirname(os.getcwd()))

import qudipy as qd
import qudipy.potential as pot
import qudipy.exchange as exch

import numpy as np
import matplotlib.pyplot as plt
import copy
from scipy.io import loadmat    # for loading Matlab files
from scipy.interpolate import interp1d

from tqdm import tqdm
import time
import math


import gc   # for memory cleanup

Specifying the dot parameters

In [ ]:
csts = qd.Constants('Si/SiO2')

nItinOrbs = [2] #[2, 4, 6, 8] #[10,12,2,4,]
n_dot_seps = 10

dot_seps = np.linspace(7.5,15, n_dot_seps) * 1E-9;
a = 6.0E-9;     # dot_width
dx_dy = 0.5;

omega = csts.hbar / (csts.me * a ** 2 )
print('omega = {:e} Hz'.format(omega))

Creating an array of grid parameters objects (for future simulations)

In [ ]:
ngridx = 200
ngridy = 200

xgrids = np.linspace( - dot_seps - 5 * a, dot_seps + 5 * a, ngridx).T
ygrids = np.tile(np.linspace( - 5 * a, 5 * a, ngridy), (n_dot_seps, 1))

all_gparams = (pot.GridParameters(xgrids[ idx], ygrids[ idx])
                    for idx in range(n_dot_seps))

# numbers of harmonic orbitals
n_xy_ho = (12,12)
xgrids.shape

Defining a function that creates a quartic potential

In [ ]:
def create_quartic(gparams, d):
    return 1 / 2 * csts.me * omega ** 2 * (
                    1 / ( 4 * d **2) * (gparams.x_mesh ** 2 - d ** 2) ** 2
                        + gparams.y_mesh ** 2
    )

Iterate over dot separations and numbers of single-electron orbitals to calculate the exchange values

In [ ]:
#%%script false

exchanges = []

omega_guess = 0.025 * csts.hbar / (csts.me * csts.a_B**2)

for n_se_orbs in nItinOrbs:
    gparams = next(all_gparams)
    exs = np.zeros((n_dot_seps,))
    
    for idx in tqdm(range(n_dot_seps)):
        gparams.update_potential(create_quartic(gparams, dot_seps[idx]))
        ens, __ = qd.qutils.solvers.solve_many_elec_SE(gparams, n_elec=2, 
                      n_xy_ho=n_xy_ho, 
                      n_se=n_se_orbs, 
                      n_sols=4,
                      consts=csts,
                      optimize_omega=False,
                      omega=omega_guess,
                      cme_dir='..\\tutorials\\QuDiPy tutorial data\\CMEs',
                      spin_subspace=[0]);
        exs[idx] = ens[1] - ens[0]
        print('exchange value is: ', exs[idx] ,
        '\nenergies are: ', ens)

        # updating omega guess
        omega_guess, __ = exch.optimize_HO_omega(gparams, 
                                             nx=n_xy_ho[0], ny=n_xy_ho[1],
                                             omega_guess=omega_guess,
                                             n_se_orbs=n_se_orbs, 
                                             opt_tol = 1e-4,
                                             consts=csts)
    exchanges.append(exs)


# Loading and plotting the exchange data from Matlab


In [ ]:

mat_contents = loadmat('N=2_4_6_8_10_M=15')
mat_contents.keys()


In [ ]:
mat_exchanges = mat_contents['exchange']
mat_dot_seps = np.linspace(7.5,15,100) * 2
mat_exchanges.T[:,:10]


Adding the exchange values from Python code


In [ ]:

%matplotlib widget
legends = ('N=2','N=4','N=6','N=8','N=10', 'N=12', 'N=2 data','N=4 data','N=6 data', 'N=8 data', )

plt.figure(figsize=(6,5))
for ex in mat_exchanges.T:
    plt.plot(mat_dot_seps, ex * 42.7E3)
#plt.legend(plot, legends)

# plotting Python results
for ex_python in exchanges:
    plt.plot(np.linspace(15,30, n_dot_seps), ex_python*1.6e25, 'o' )
plt.xscale('linear')
plt.yscale('log')
plt.legend(legends, loc='best', fancybox=True, ncol=2)
plt.xlim((15,30))

plt.show()
